In [ ]:
from pathlib import Path
import face_alignment
from skimage import io


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import collections
from dataclasses import dataclass
matplotlib.rcParams['figure.dpi'] = 300


In [ ]:
# BASE_DIR = Path(__file__).parent.parent
# BASE_DIR = Path.cwd().
DRIVE_NAME = "F:\\"
DATA_DIR = Path(DRIVE_NAME).joinpath('data')
IMAGE_DIR = DATA_DIR.joinpath('images')
DUMP_DIR = DATA_DIR.joinpath('dump')
MODEL_DIR = DATA_DIR.joinpath('models')
# for c in DATA_DIR.iterdir(): print(c)

test_dir = IMAGE_DIR.joinpath('2010-09-27')

fa = face_alignment.FaceAlignment(
    face_alignment.LandmarksType._3D,
    face_detector='sfd',
    flip_input=False,
    device='cpu',
)

@dataclass
class Predtype:
    list_slice: slice
    color: tuple

pred_types = {
    'face': Predtype(slice(0, 17), (0.682, 0.780, 0.909, 0.5)),
    'eyebrow1': Predtype(slice(17, 22), (1.0, 0.498, 0.055, 0.4)),
    'eyebrow2': Predtype(slice(22, 27), (1.0, 0.498, 0.055, 0.4)),
    'nose': Predtype(slice(27, 31), (0.345, 0.239, 0.443, 0.4)),
    'nostril': Predtype(slice(31, 36), (0.345, 0.239, 0.443, 0.4)),
    'eye1': Predtype(slice(36, 42), (0.596, 0.875, 0.541, 0.3)),
    'eye2': Predtype(slice(42, 48), (0.596, 0.875, 0.541, 0.3)),
    'lips': Predtype(slice(48, 60), (0.596, 0.875, 0.541, 0.3)),
    'teeth': Predtype(slice(60, 68), (0.596, 0.875, 0.541, 0.4))
}


In [ ]:
img_preds = fa.get_landmarks_from_directory(str(test_dir))

In [ ]:
# 2D-Plot
plot_style = dict(
    marker='.',
    markersize=1,
    linestyle='none',
    # linewidth=2,
)



In [ ]:
for img_path, faces in img_preds.items():
    # print(img_path)
    # print(len(value))
    # print("\n")
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.imshow(io.imread(img_path))
    ax.axis('off')
    for index, face in enumerate(faces):
        # ax = fig.add_subplot(1, len(faces), index + 1)
        for pred_type in pred_types.values():
            # print(pred)
            # print(type(pred))
            
            ax.plot(
                face[pred_type.list_slice, 0],
                face[pred_type.list_slice, 1],
                color=pred_type.color,
                **plot_style,
            )

        

In [ ]:
# 3D-Plot

for img_path, faces in img_preds.items():
    # print(img_path)
    # print(len(value))
    # print("\n")

    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1], projection='3d')
    # ax.imshow(io.imread(img_path))
    # ax.axis('off')
    for index, face in enumerate(faces):
        # ax = fig.add_subplot(1, len(faces), index + 1)
        surf = ax.scatter(
            face[:, 0] * 1.2,
            face[:, 1],
            face[:, 2],
            c='cyan',
            alpha=1.0,
            edgecolor='b',
        )
        for pred_type in pred_types.values():
            # print(pred)
            # print(type(pred))
            ax.plot3D(
                face[pred_type.list_slice, 0] * 1.2,
                face[pred_type.list_slice, 1],
                face[pred_type.list_slice, 2],
                color='blue',
            )
        ax.view_init(elev=45., azim=45.)
        ax.set_xlim(ax.get_xlim()[::-1])
        plt.show()



# ax = fig.add_subplot(1, 2, 2, projection='3d')
# surf = ax.scatter(face[:, 0] * 1.2,
#                   face[:, 1],
#                   face[:, 2],
#                   c='cyan',
#                   alpha=1.0,
#                   edgecolor='b',
# )


# for pred_type in pred_types.values():
#     ax.plot3D(face[pred_type.slice, 0] * 1.2,
#               face[pred_type.slice, 1],
#               face[pred_type.slice, 2], color='blue')

# ax.view_init(elev=90., azim=90.)
# ax.set_xlim(ax.get_xlim()[::-1])
# plt.show()

In [ ]:
# OLD BOUNDING BOX THINGS
images_list = []

embeddings_list = []
# index = 0
for image_path in test_dir.iterdir():
    image_dict = {
        'path': image_path,
        'face_list': [],
    }
#     print(image_path)
    # index += 1
    image = cv2.imread(str(image_path))
#     print(image.shape)
    
    faces = mtcnn_face_detector.detect_faces(image)
#     print(len(faces))
    face_index = 0
    for face_dict in faces:
        
        
        face_index += 1
        confidence = face_dict['confidence']
#         print(confidence)
        if confidence < 0.5:
            continue
        (x, y, width, height) = face_dict['box']
        
        
        
        # Get face image from bounding box and preprocess
        face_image = image[y:y + height, x:x + width]
        face_image = cv2.resize(face_image, (224, 224))
        face_image_window = '{}'.format(face_index)
        cv2.imshow(face_image_window, face_image)
        cv2.moveWindow(face_image_window, 224*(face_index - 1), 0)
        
        # Draw bounding box
        image = cv2.rectangle(
            image,
            (x, y),
            (x + width, y + height),
            (0, 255, 0),
            2,
        )
        
        samples = preprocess_input(np.expand_dims(face_image.astype('float32'), axis=0), version=2)
        embedding = vgg_embedding_extractor.predict(samples)
        
        face_data_dict = {
            'source_image_path': image_path,
            'mtcnn_extraction': face_dict,
            'vgg_extraction': embedding,
        }
        image_dict['face_list'].append(face_data_dict)
    images_list.append(image_dict)
    cv2.imshow('test image', image)
    cv2.moveWindow('test image', 0, 224)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
    



In [ ]:
with open(test_dir.joinpath(test_dir.name + '.pkl'), 'wb') as save_path:
    pickle.dump(images_list, save_path)
    
    
# np.linalg.norm(embeddings_list[0] - embeddings_list[0], ord=2)
# cosine(embeddings_list[6], embeddings_list[7])

# with open(test_dir.joinpath(test_dir.name + '.pkl'), 'rb') as load_path:
#     asdf = pickle.load(load_path)